## Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List, Dict, Tuple
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
import statsmodels.api as sm
from scipy.stats import spearmanr
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
import holidays
from matplotlib.dates import WeekdayLocator
import matplotlib.dates as mdates
from sklearn.linear_model import Ridge
import sys
from prophet import Prophet
sys.path.append('/Users/i.nuriev/repository/mmm-tools/mmm_tools')
from media_transformation import *
from model_building import build_models
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'media_transformation'

### Input Data

**Источники данных**

**Ключевые метрики для бизнеса: DAU и TimeSpent** 

Данные по DAU Одноклассников и ее конкурентов были собраны из MediaScope:
Конкуренты Одноклассников - это соц.сети и мессенджеры:
- dau_telegram_35_plus - активные пользователи telegram в возрастной категории 35+
- dau_dzen_35_plus - активные пользователи Дзена в возрастной категории 35+
- dau_dzen_news_web_35_plus - активные пользователи новостного сайта Дзена в возрастной категории 35+
- dau_facebook_35_plus - активные пользователи Facebook в возрастной категории 35+
- dau_instagram_35_plus - активные пользователи Instagram в возрастной категории 35+
- dau_ok_35_plus - активные пользователи OK в возрастной категории 35+ (целевая переменная)
- dau_vkontakte_35_plus - активные пользователи Вконтакте в возрастной категории 35+
- dau_whatsapp_35_plus - активные пользователи Whatsapp в возрастной категории 35+
- dau_youtube_35_plus - активные пользователи Youtube в возрастной категории 35+
- dau_tiktok_35_plus - активные пользователи Tiktok в возрастной категории 35+
- dau_runet_35_plus - активные пользователи всего Рунета в возрастной категории 35+
- dau_meta_35_plus - активные пользователи всех проектов Meta в возрастной категории 35+

Далее некоторые факторы были отброшены

Данные из MyTracker:
- returns_u_35_plus - Это пользователи 35 + , которые бездействовали в течение заданного интервала времени (окно неактивности для возвратов), но проявили активность в выбранный период отчёта.
Окно неактивности для возвратов по умолчанию: 30 дней
- returns_u_total - Это все пользователи, которые бездействовали в течение заданного интервала времени (окно неактивности для возвратов), но проявили активность в выбранный период отчёта.
Окно неактивности для возвратов по умолчанию: 30 дней
- installs_d_35_plus - Количество установок приложения пользователями 35 +, сделанных за период отчёта.
- installs_d_total - Количество установок приложения всеми пользователями, сделанных за период отчёта.

## **Предобработка BHT-метрик**

**дезагрегирование недельных (или месячных) данных по дням**

In [ ]:
df_bht = pd.read_excel('../actual_data.xlsx',sheet_name='bht',header=4)

In [ ]:
# ставим дату в индекс
df_bht.index = df_bht.date_start

**Вставляем последнюю дату из date_end в качестве последней строки, потому что дизагрегирование будет происходить по date_start и послядняя дата date_end не входит в индекс, соответственно одну неделю мы потеряем.**

In [ ]:
last_row = df_bht.iloc[[-1]] # дублируем последнюю строку
last_row.index = [df_bht.date_end.max()]
df_bht = pd.concat([df_bht,last_row])

In [ ]:
# дезагрегация по дням
df_bht = df_bht.asfreq('D').interpolate(method='linear')

In [ ]:
df_bht = df_bht[['aa_prompted_kpi_percent_ttl_roll',
       'ba_spontaneous_kpi_percent_ttl_roll',
       'consid_kpi_percent_ttl_roll']].reset_index().rename(columns={'index':'dt'})

## **Предобработка данных WordStat**

**Статистика по поисковому слову "Одноклассники"**

In [ ]:
ws_weekly = pd.read_excel('../wordstat_dynamic.xlsx',sheet_name='weekly')
ws_weekly.rename(columns={'Week from':'dt'},inplace=True)
ws_weekly.index = ws_weekly.dt

In [ ]:
ws_weekly = ws_weekly.asfreq('D').interpolate(method='linear')

In [ ]:
ws_daily = pd.read_excel('../wordstat_dynamic.xlsx',sheet_name='daily')
ws_daily.rename(columns={'Date':'dt'},inplace=True)
ws_daily.index = ws_daily.dt

In [ ]:
ws_weekly = ws_weekly[ws_weekly.dt.lt(ws_daily.dt.min())]
ws_ok = pd.concat([ws_weekly,ws_daily])

In [ ]:
ws_ok.rename(columns={'Number of queries':'wordstat_num_of_queries',
                      'Percentage of total queries, %':'wordstat_perc_of_total_queries'},
            inplace=True)

In [ ]:
ws_ok.reset_index(drop=True,inplace=True)

## **Предобработка блогеров**

In [ ]:
df = pd.read_excel('../actual_data.xlsx',
                   sheet_name='dau')

In [ ]:
# применим линейную интерполяцию для блогеров для одного периода
# df.loc[df.dt.ge('2023-04-10')&df.dt.le('2023-04-24'),'bloggers'] = df[df.dt.ge('2023-04-10')&df.dt.le('2023-04-24')].bloggers.interpolate(method='linear')

### **Целевая переменная**

$$\widehat{DAU_t^{ok}} = \frac{DAU_t^{ok}}{DAU_t^{runet}}$$

In [ ]:
df.dt = pd.to_datetime(df.dt)

In [ ]:
df['dau_ok_35_plus_relative'] = df['dau_ok_35_plus']/df['dau_runet_35_plus']

In [ ]:
df = df.merge(ws_ok, how='left', on='dt')
df = df.merge(df_bht,how='left', on='dt') # данные с 2022-09-19

In [ ]:
df.index = df.dt

In [ ]:
df.ba_spontaneous_kpi_percent_ttl_roll=df.ba_spontaneous_kpi_percent_ttl_roll.ffill()

**Добавим колонку с праздниками**

In [ ]:
holid = holidays.Russia()
df['dummy_holidays'] = df.dt.apply(lambda x: 1 if x in holid else 0)

# 1-ое сентября тоже праздник и в данных наблюдается всплеск
df.loc[df.index == '2022-09-01','dummy_holidays'] = 1
df.loc[df.index == '2023-09-01','dummy_holidays'] = 1

# Православная Пасха
df.loc[df.index == '2023-04-16','dummy_holidays'] = 1
df.loc[df.index == '2024-05-05','dummy_holidays'] = 1

# Добавим единицы на выходные дни, пришедшиеся на будни из-за праздников 
df.loc[(df.index >= '2024-04-29')&(df.index <= '2024-05-01') ,'dummy_holidays'] = 1
df.loc[(df.index >= '2024-05-09')&(df.index <= '2024-05-10') ,'dummy_holidays'] = 1

In [ ]:
print(f'Минимальная дата: {df.dt.min()}\nМаксимальная дата: {df.dt.max()}')

**Список целевых переменных**
- Возврат (u) [beta]_total - количество всех пользователей, которые проявили активность в этот день, при этом 30 + дней до этого были неактивны, шт
- Возврат (u) [beta]_35_plus - количество пользователей с возрастом 35+, которые проявили активность в этот день, при этом 30 + дней до этого были неактивны, шт
- Установки (d)_total - тотальные установки из MyTracker всех юзеров из РФ
-  Установки (d)_35_plus - Количество установок приложения пользователями 35 +, сделанных за период отчёта.
-  dau_ok_ttl - DAU для людей возрастом 35+ MediaScope, в миллионах
-  dau_ok_35_plus - все активные пользователи по дням в миллионах
-  tdd_ok_35_plus - времяпровождение в миллионах минут для 35 + в ОК
-  tdd_ok_ttl - общее времяпровождение в миллионах минут в ОК

**Медиа-переменные**
-  digital_media_ots - показы в интернете, ots
-  performance_ots - показы контекстной рекламы, ots
-  bloggers_ots - показы у блогеров в YouTube/Telegram
-  tv_ots - просмотры в телевизоре, ots в штуках. (Бывает в штуках или в тысячах штук)
-  ooh_ots - наружная реклама, ots
-  radio_ots - радио, ots

### Data quality

In [ ]:
df.describe()

- Среднее количество активных пользователей в ОК составляет более 20 млн.в день
- TimeSpent приведен в выражении в миллионах минут

In [ ]:
duplicates = df.duplicated(subset = ['dt']).sum()

In [ ]:
print(f'Размерность выборки: {df.shape}')
print(f'Количество дубликатов по ключу (в датах): {duplicates}\nПропуски в данных:')
df.isna().sum()

impressions (для интернета - способ технической регистрации) = ots

In [ ]:
# df.OOH = df.Metro + df.OOH + df.Radio

In [ ]:
df.fillna(0,inplace=True)

**Data types**

In [ ]:
df.dtypes

In [ ]:
# Переместим дату в индекс и удалим 
df.index = df.dt
df.drop(columns=['dt'],inplace=True)

In [ ]:
kpi = 'dau_ok_35_plus_relative'

## **Exloratory Data Analyses**

Вспомогательная строчка для отрисовки интерактивных графиков

In [ ]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

**Plot variables**

In [ ]:
plt.figure(figsize = (20,12))
# kpi
plt.scatter(df.index, df[kpi], label=kpi)
# вычисляем тренд
horizont = range(len(df.index))
coefs = np.polyfit(horizont, df[kpi], 1) # кф-ты тренда
trend = np.poly1d(coefs) 
plt.plot(np.array(df.index), trend(horizont), "r--", label='Тренд')
plt.xlabel('Дата',fontsize=20)
plt.ylabel(kpi,fontsize=20)
plt.grid(True)
plt.legend(fontsize=20)
# Отображаем график
plt.show()

In [ ]:
df[kpi].iplot()

### **Выделим сезонность в данных**

**Prophet**

In [ ]:
data = pd.read_excel('../actual_data.xlsx', sheet_name='seasonal')
data['y'] = data.dau_ok_35_plus/data.dau_runet_35_plus
data.rename(columns={'dt':'ds'},inplace=True)

In [ ]:
df_holidays = pd.DataFrame({
  'holiday': 'dummy_holidays',
  'ds': pd.to_datetime(df.dummy_holidays.eq(1).index),
  'lower_window': 0,
  'upper_window': 1,
})

In [ ]:
# data['cap'] = trend(horizont)

In [ ]:
model = Prophet(growth='linear',
                seasonality_mode='additive', 
                weekly_seasonality=True, 
                yearly_seasonality=2, # 2 гармоники 
                holidays=df_holidays,
               seasonality_prior_scale=30)
model.fit(data)

future = model.make_future_dataframe(periods=0)

# future['cap'] = trend(horizont)

forecast = model.predict(future)

# Визуализация результатов
fig = model.plot(forecast)

In [ ]:
forecast = forecast[:len(data)]

In [ ]:
# Визуализация результатов
fig, ax = plt.subplots(figsize=(15, 7))
ax.plot(data['ds'], data['y'], label='Исходный ряд', color='blue')
ax.plot(data['ds'], forecast['trend'], label='Тренд', color='green')
ax.plot(data['ds'], forecast['weekly'], label='Сезонность (недельная)', color='orange')
ax.plot(data['ds'], forecast['yearly'], label='Сезонность (годовая)', color='black')
# ax.plot(data['ds'], forecast['holiday'], label='праздники', color='red')
plt.legend()
plt.grid(True)
plt.title('Выделение сезонности и тренда с помощью Prophet')
plt.xticks(fontsize=12)
plt.ylabel('Относительное DAU OK',fontsize=14)
ax.xaxis.set_major_locator(mdates.MonthLocator())
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
forecast = forecast[forecast.ds.ge('2022-08-01')].reset_index(drop=True)

In [ ]:
forecast.index = df.index
df['seasonality_weekly_dau'] = forecast['weekly']
df['seasonality_yearly_dau'] = forecast['yearly']
df['trend_linear_dau'] = trend(horizont) 
df['trend_prophet_dau'] = forecast['trend']

**Построим рассчитаем корреляции факторов с целевой переменной**

Так как для решения нашей задачи мы бы хотели обучить линейную регрессию, было бы хорошо найти признаки, "наиболее линейно" связанные с целевой переменной, иначе говоря, посмотреть на коэффициент корреляции Пирсона между признаками и целевой переменной.

In [ ]:
# X = df.drop(columns=kpi_all_colls)

# for i in kpi_DAU:
#     corr = X.corrwith(df[i]).sort_values(ascending=False)
#     sns.barplot(y=corr.index, x=corr)
#     plt.title(i,fontsize=15)
#     plt.xlabel('Корреляция')
#     plt.ylabel('Факторы')
#     plt.grid(True)
#     plt.show()

Radio, OOH, Ditital_impr - отрицательно коррелируют с DAU. Попробуем медиа-трансформации

## **Стандартизация переменных**

In [ ]:
media_factors = ['tv_ots', 'metro_ots', 'ooh_ots', 'radio_ots','bloggers_ots','digital_media_ots','performance_ots']
competitors = ['dau_meta_35_plus', 'dau_telegram_35_plus',
       'dau_dzen_35_plus', 'dau_dzen_news_web_35_plus', 'dau_facebook_35_plus',
       'dau_instagram_35_plus',
       'dau_whatsapp_35_plus', 'dau_youtube_35_plus',
       'dau_tiktok_35_plus'] # Cross Web
dummy_variables = ['dummy_holidays','dummy_show_bur']
other_factors = ['seasonality_weekly_dau','trend_linear_dau',
                 'trend_prophet_dau','seasonality_yearly_dau'] # Prophet
ws_factors = ['wordstat_num_of_queries', 'wordstat_perc_of_total_queries']
bht_factors = ['aa_prompted_kpi_percent_ttl_roll',
       'ba_spontaneous_kpi_percent_ttl_roll', 'consid_kpi_percent_ttl_roll']
non_media_factors = competitors + dummy_variables + bht_factors + ws_factors + other_factors + ['product_performance_ots','product_activity_piar_ots']

In [ ]:
numeric_data = df[media_factors+competitors+other_factors + bht_factors + ws_factors + ['product_performance_ots','product_activity_piar_ots']] #.select_dtypes([np.number, float]).copy()
numeric_features = numeric_data.columns

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[numeric_features])
scaled_df = pd.DataFrame(scaled_data,columns=numeric_features)
scaled_df.index = df.index
scaled_df['dummy_holidays'] = df['dummy_holidays']
scaled_df['dummy_show_bur'] = df['dummy_show_bur']

**Media-transformation**

In [ ]:
media_configs_file_path = '/Users/i.nuriev/repository/MMM/ok/media_configs_ok_initial.xlsx'

In [ ]:
df_params = upload_media_params_file(media_configs_file_path)

In [ ]:
init_params = prepare_media_params_init(df_params)

In [ ]:
params_init_dict = create_media_params_dict(params=init_params['params'], 
                                       media_factors=init_params['media'])

## **Для каждой РК сделаем свои медиа-трансформации**

**Разделим данные на 4 периода и подберем для каждой РК свои параметры медиа-преобразования:**
- c 2022-08-01 по 2022-12-31
- c 2023-01-01 по 2023-05-19
- с 2023-05-20 по 2023-12-31
- c 2024-01-01 по 2024-05-01

In [ ]:
df_first_rk = df.loc[df.index <= '2022-12-31']
df_second_rk = df.loc[(df.index >= '2023-01-01')&(df.index <= '2023-05-19')]
df_third_rk = df.loc[(df.index > '2023-05-19')&(df.index <= '2023-12-31')]
df_fourth_rk = df.loc[df.index > '2023-12-31']

scaled_df_first_rk = scaled_df.loc[scaled_df.index <= '2022-12-31']
scaled_df_second_rk = scaled_df.loc[(scaled_df.index >= '2023-01-01')&(scaled_df.index <= '2023-05-19')]
scaled_df_third_rk = scaled_df.loc[(df.index > '2023-05-19')&(df.index <= '2023-12-31')]
scaled_df_fourth_rk = scaled_df.loc[df.index > '2023-12-31']

scaled_df_first_rk[kpi] = df_first_rk[kpi]
scaled_df_second_rk[kpi] = df_second_rk[kpi]
scaled_df_third_rk[kpi] = df_third_rk[kpi]
scaled_df_fourth_rk[kpi] = df_fourth_rk[kpi]

# Обучить ридж-регрессию на всех факторах и выбрать из них наиболее релевантные

In [ ]:
non_media_final_factors = ['dau_dzen_35_plus',
                           'dummy_holidays',
                           'dau_youtube_35_plus',
                           'product_performance_ots',
                           'product_activity_piar_ots',
                           #'dummy_show_bur',
                           #'dau_instagram_35_plus',
                           #'seasonality_weekly_dau',
                           # 'trend_linear_dau',
                           'seasonality_yearly_dau',
                           'wordstat_num_of_queries', 
                           # 'wordstat_perc_of_total_queries',
                           # 'aa_prompted_kpi_percent_ttl_roll',
                           'ba_spontaneous_kpi_percent_ttl_roll',]
                           # 'consid_kpi_percent_ttl_roll']

model_factors = non_media_final_factors+media_factors

In [ ]:
# ridge-регрессия
model = Ridge()
model.fit(scaled_df[model_factors], df[kpi])
y_model = model.predict(scaled_df[model_factors])

In [ ]:
def show_weights(features, weights, scales):
    fig, axs = plt.subplots(figsize=(14, 10), ncols=2)
    sorted_weights = sorted(zip(weights, features, scales), reverse=True)
    weights = [x[0] for x in sorted_weights]
    features = [x[1] for x in sorted_weights]
    scales = [x[2] for x in sorted_weights]
    sns.barplot(y=features, x=weights, ax=axs[0])
    axs[0].set_xlabel("Weight")
    sns.barplot(y=features, x=scales, ax=axs[1])
    axs[1].set_xlabel("Scale")
    plt.tight_layout()

In [ ]:
scales = pd.Series(data=scaled_df[model_factors].std(axis=0), index=scaled_df[model_factors].columns.values)
show_weights(scaled_df[model_factors].columns.values, model.coef_, scales)

**Обучим линейную регрессию**

In [ ]:
lag_value = 1 # количество лагов для вычисления HAC оценки
y = df[kpi]
X = sm.add_constant(scaled_df[model_factors])
model = sm.OLS(y,X)
result = model.fit(cov_type='HAC', cov_kwds={'maxlags': lag_value})
print(result.summary())

### **Оптимизация со штрафом за отрицательные коэффициенты перед медиа-факторами**

In [ ]:
df_params = upload_media_params_file(media_configs_file_path)
init_params = prepare_media_params_init(df_params)
params_init_dict = create_media_params_dict(params=init_params['params'], 
                                            media_factors=init_params['media'])

In [ ]:
opt_params_first_rk = optimize_media_params_penalty(y=df_first_rk[kpi],
                      df=scaled_df_first_rk[media_factors],
                      media_factors=init_params['media'],
                      params_init=init_params['params'],
                      bounds_init=init_params['bounds'],
                      tol=0.000000001,
                      epsilon_step=0.01
                     )
opt_params_second_rk = optimize_media_params_penalty(y=df_second_rk[kpi],
                      df=scaled_df_second_rk[media_factors],
                      media_factors=init_params['media'],
                      params_init=init_params['params'],
                      bounds_init=init_params['bounds'],
                      tol=0.000000001,
                      epsilon_step=0.01
                     )
opt_params_third_rk = optimize_media_params_penalty(y=df_third_rk[kpi],
                      df=scaled_df_third_rk[media_factors],
                      media_factors=init_params['media'],
                      params_init=init_params['params'],
                      bounds_init=init_params['bounds'],
                      tol=0.000000001,
                      epsilon_step=0.01
                     )
opt_params_fourth_rk = optimize_media_params_penalty(y=df_fourth_rk[kpi],
                      df=scaled_df_fourth_rk[media_factors],
                      media_factors=init_params['media'],
                      params_init=init_params['params'],
                      bounds_init=init_params['bounds'],
                      tol=0.000000001,
                      epsilon_step=0.01
                     )

In [ ]:
optimal_params_dict_first_rk = create_media_params_dict(params=opt_params_first_rk, 
                         media_factors=init_params['media'])
optimal_params_dict_second_rk = create_media_params_dict(params=opt_params_second_rk, 
                         media_factors=init_params['media'])
optimal_params_dict_third_rk = create_media_params_dict(params=opt_params_third_rk, 
                         media_factors=init_params['media'])
optimal_params_dict_fourth_rk = create_media_params_dict(params=opt_params_fourth_rk, 
                         media_factors=init_params['media'])

In [ ]:
optimal_params_dict_third_rk['performance_ots']['gamma'] = 0.8
optimal_params_dict_third_rk['performance_ots']['beta'] = 1.2



optimal_params_dict_third_rk['radio_ots']['beta'] = 1.1
optimal_params_dict_third_rk['radio_ots']['alpha'] = 0.9
optimal_params_dict_second_rk['radio_ots']['alpha'] = 1.1

optimal_params_dict_second_rk['ooh_ots']['beta'] = 1.2

optimal_params_dict_third_rk['ooh_ots']['alpha'] = 2.5
optimal_params_dict_third_rk['ooh_ots']['beta'] = 1.2

optimal_params_dict_fourth_rk['ooh_ots']['alpha'] = 1.6
optimal_params_dict_fourth_rk['ooh_ots']['gamma'] = 1.1
optimal_params_dict_fourth_rk['ooh_ots']['beta'] = 1.2

optimal_params_dict_third_rk['bloggers_ots']['alpha'] = 0.8
optimal_params_dict_third_rk['bloggers_ots']['beta'] = 1.2

optimal_params_dict_second_rk['digital_media_ots']['alpha'] = 1.1
optimal_params_dict_third_rk['digital_media_ots']['alpha'] = 1.1
optimal_params_dict_second_rk['digital_media_ots']['decay'] = 0.4
optimal_params_dict_third_rk['digital_media_ots']['decay'] = 0.4
optimal_params_dict_second_rk['digital_media_ots']['beta'] = 0.4
optimal_params_dict_third_rk['digital_media_ots']['beta'] = 0.4
optimal_params_dict_second_rk['digital_media_ots']['gamma'] = 1
optimal_params_dict_third_rk['digital_media_ots']['gamma'] = 1


optimal_params_dict_fourth_rk['digital_media_ots']['alpha'] = 1.9

In [ ]:
sorted_inital_dict = {key: params_init_dict[key] for key in media_factors}
show_comparison(media_params_optimal=optimal_params_dict_first_rk,
                media_params_init=sorted_inital_dict)

In [ ]:
show_comparison(media_params_optimal=optimal_params_dict_second_rk,
                media_params_init=sorted_inital_dict)

In [ ]:
show_comparison(media_params_optimal=optimal_params_dict_third_rk,
                media_params_init=sorted_inital_dict) 

In [ ]:
show_comparison(media_params_optimal=optimal_params_dict_fourth_rk,
                media_params_init=sorted_inital_dict) 

In [ ]:
transform_df_first_rk = transform_media_data(scaled_df_first_rk,optimal_params_dict_first_rk,media_factors)
transform_df_second_rk = transform_media_data(scaled_df_second_rk,optimal_params_dict_second_rk,media_factors)
transform_df_third_rk = transform_media_data(scaled_df_third_rk,optimal_params_dict_third_rk,media_factors)
transform_df_fourth_rk = transform_media_data(scaled_df_fourth_rk,optimal_params_dict_fourth_rk,media_factors)

In [ ]:
df_transf = pd.concat([transform_df_first_rk,transform_df_second_rk,transform_df_third_rk,transform_df_fourth_rk]) # медиа-факторы
df_final = pd.concat([df_transf, scaled_df[non_media_factors],df[kpi]],axis=1) # контрольные факторы

In [ ]:
# plot_saturation_points(media_factors, optimal_params_dict_first_rk,scaled_df_first_rk,'saturate_power',
#                       sorted_inital_dict)

In [ ]:
# plot_saturation_points(media_factors, optimal_params_dict_second_rk,scaled_df_second_rk,'saturate_power',
#                       sorted_inital_dict)

In [ ]:
# plot_saturation_points(media_factors, optimal_params_dict_third_rk,scaled_df_third_rk,'saturate_power',
#                       sorted_inital_dict)

In [ ]:
# plot_saturation_points(media_factors, optimal_params_dict_fourth_rk,scaled_df_fourth_rk,'saturate_power',
#                       sorted_inital_dict)

In [ ]:
plot_transformed_data(scaled_df,df_final,media_factors + ['product_activity_piar_ots','product_performance_ots'])

In [ ]:
df_final=pd.concat([df_final,df.dau_runet_35_plus],axis=1)

In [ ]:
df_final.to_csv('df_final_dau.csv')

In [ ]:
pd.DataFrame(optimal_params_dict_first_rk).to_excel('best_params_first_rk.xlsx')

In [ ]:
pd.DataFrame(optimal_params_dict_second_rk).to_excel('best_params_second_rk.xlsx')

In [ ]:
pd.DataFrame(optimal_params_dict_third_rk).to_excel('best_params_third_rk.xlsx')

In [ ]:
pd.DataFrame(optimal_params_dict_fourth_rk).to_excel('best_params_fourth_rk.xlsx')